In [1]:
import pandas as pd
import pyodbc
import datetime

## Import data from SQL Server

In [2]:
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=.;DATABASE=sakila;Trusted_Connection=yes') 

In [3]:
query = "SELECT * FROM rental"
rental = pd.read_sql(query, conn)
print(rental.shape)

query = "SELECT * FROM payment"
payment = pd.read_sql(query, conn)
print(payment.shape)

query = "SELECT * FROM customer"
customer = pd.read_sql(query, conn)
print(customer.shape)

query = "SELECT * FROM address ad JOIN city c ON c.city_id=ad.city_id \
                                JOIN country co ON co.country_id = c.country_id"
address = pd.read_sql(query, conn)
print(address.shape)

C:\Users\buidu\AppData\Local\Temp\ipykernel_17316\3203878691.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rental = pd.read_sql(query, conn)


(16044, 7)


C:\Users\buidu\AppData\Local\Temp\ipykernel_17316\3203878691.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  payment = pd.read_sql(query, conn)


(16049, 7)
(599, 9)
(603, 15)


C:\Users\buidu\AppData\Local\Temp\ipykernel_17316\3203878691.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer = pd.read_sql(query, conn)
C:\Users\buidu\AppData\Local\Temp\ipykernel_17316\3203878691.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  address = pd.read_sql(query, conn)


In [4]:
rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1346,2005-06-15 12:39:52,4540,585,2005-06-24 17:43:52,1,2006-02-15 21:30:53
1,1347,2005-06-15 12:43:43,374,190,2005-06-16 09:55:43,1,2006-02-15 21:30:53
2,1348,2005-06-15 12:45:30,2078,196,2005-06-17 17:12:30,1,2006-02-15 21:30:53
3,1349,2005-06-15 12:49:02,1131,267,2005-06-17 15:20:02,1,2006-02-15 21:30:53
4,1350,2005-06-15 12:50:25,4261,316,2005-06-23 11:35:25,1,2006-02-15 21:30:53


In [5]:
payment.head()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,1,1,1,76.0,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
1,2,1,1,573.0,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30
2,3,1,1,1185.0,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30
3,4,1,2,1422.0,0.99,2005-06-15 18:02:53,2006-02-15 22:12:30
4,5,1,2,1476.0,9.99,2005-06-15 21:08:46,2006-02-15 22:12:30


In [6]:
customer.head()

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20
3,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [7]:
address = address[['address_id','city','country']]
address.tail()

,address_id,city,country
598,601,Jinzhou,China
599,602,Patras,Greece
600,603,Sullana,Peru
601,604,Lausanne,Switzerland
602,605,Tieli,China


## Integrate data for dimension and fact tables

* Rental Dimension

In [8]:
dimRental = rental.merge(right=payment, on='rental_id', how='inner', 
                         suffixes=('', '_ren')).drop(columns=['customer_id_ren','staff_id_ren','last_update_ren','last_update','payment_id'])

In [9]:
dimRental['rental_date'] = dimRental['rental_date'].apply(lambda x: x.date())
dimRental['rental_month'] = dimRental['rental_date'].apply(lambda x: x.month)
dimRental['return_date'] = dimRental['return_date'].apply(lambda x: x.date())
dimRental['payment_date'] = dimRental['payment_date'].apply(lambda x: x.date())

In [10]:
print(dimRental.info())
dimRental.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16044 entries, 0 to 16043
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   rental_id     16044 non-null  int64  
 1   rental_date   16044 non-null  object 
 2   inventory_id  16044 non-null  int64  
 3   customer_id   16044 non-null  int64  
 4   return_date   15861 non-null  object 
 5   staff_id      16044 non-null  int64  
 6   amount        16044 non-null  float64
 7   payment_date  16044 non-null  object 
 8   rental_month  16044 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 1.1+ MB
None


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,amount,payment_date,rental_month
0,1346,2005-06-15,4540,585,2005-06-24,1,7.99,2005-06-15,6
1,1347,2005-06-15,374,190,2005-06-16,1,2.99,2005-06-15,6
2,1348,2005-06-15,2078,196,2005-06-17,1,2.99,2005-06-15,6
3,1349,2005-06-15,1131,267,2005-06-17,1,4.99,2005-06-15,6
4,1350,2005-06-15,4261,316,2005-06-23,1,4.99,2005-06-15,6


In [23]:
dimRental[dimRental['rental_date'].apply(lambda x: x.month == 2)]

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,amount,payment_date,rental_month
10147,11496,2006-02-14,2047,155,NaT,1,7.98,2006-02-14,2
10192,11541,2006-02-14,2026,335,NaT,1,0.99,2006-02-14,2
10214,11563,2006-02-14,1545,83,NaT,1,4.99,2006-02-14,2
10228,11577,2006-02-14,4106,219,NaT,2,4.99,2006-02-14,2
10244,11593,2006-02-14,817,99,NaT,1,0.99,2006-02-14,2
...,...,...,...,...,...,...,...,...,...
14512,15862,2006-02-14,925,215,NaT,1,0.99,2006-02-14,2
14517,15867,2006-02-14,837,505,NaT,2,4.99,2006-02-14,2
14525,15875,2006-02-14,3611,41,NaT,1,2.99,2006-02-14,2
14544,15894,2006-02-14,4416,168,NaT,1,0.99,2006-02-14,2


* Customer Dimension

In [24]:
customer['full_name'] = customer['first_name'] + ' ' + customer['last_name']
customer.drop(columns=['first_name','last_name','email','create_date','last_update'], axis=1, inplace=True)
dimCustomer = customer.merge(right=address, on='address_id', how='inner')

In [25]:
print(dimCustomer.info())
dimCustomer.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  599 non-null    int64 
 1   store_id     599 non-null    int64 
 2   address_id   599 non-null    int64 
 3   active       599 non-null    object
 4   full_name    599 non-null    object
 5   city         599 non-null    object
 6   country      599 non-null    object
dtypes: int64(3), object(4)
memory usage: 32.9+ KB
None


,customer_id,store_id,address_id,active,full_name,city,country
0,1,1,5,1,MARY SMITH,Sasebo,Japan
1,2,1,6,1,PATRICIA JOHNSON,San Bernardino,United States
2,3,1,7,1,LINDA WILLIAMS,Athenai,Greece
3,4,2,8,1,BARBARA JONES,Myingyan,Myanmar
4,5,1,9,1,ELIZABETH BROWN,Nantou,Taiwan


* Segment Fact Table

In [27]:
factSegment = dimCustomer.merge(right=dimRental, on='customer_id', how='inner', suffixes=('', '_ren'))
factSegment = factSegment[['customer_id','city','country','active','full_name','rental_id','amount','rental_date']]

print(factSegment.info())
factSegment.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16044 entries, 0 to 16043
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  16044 non-null  int64  
 1   city         16044 non-null  object 
 2   country      16044 non-null  object 
 3   active       16044 non-null  object 
 4   full_name    16044 non-null  object 
 5   rental_id    16044 non-null  int64  
 6   amount       16044 non-null  float64
 7   rental_date  16044 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 1002.9+ KB
None


,customer_id,city,country,active,full_name,rental_id,amount,rental_date
0,1,Sasebo,Japan,1,MARY SMITH,1422,0.99,2005-06-15
1,1,Sasebo,Japan,1,MARY SMITH,1476,9.99,2005-06-15
2,1,Sasebo,Japan,1,MARY SMITH,1725,4.99,2005-06-16
3,1,Sasebo,Japan,1,MARY SMITH,2308,4.99,2005-06-18
4,1,Sasebo,Japan,1,MARY SMITH,2363,0.99,2005-06-18


* Recency measure

In [28]:
data = set()
for ids, val in factSegment.groupby('customer_id').aggregate({'rental_date':'max'}).rental_date.to_dict().items():
    first_date = factSegment['rental_date'].min()
    recency = pd.Timedelta(val-first_date).days
    data.add((ids, first_date, recency))
    
m_val = pd.DataFrame(data).rename(columns={0: "customer_id", 1:"first_date", 2:"recency"})

factSegment = factSegment.merge(right=m_val, on='customer_id', how='inner')

* Monetary measure

In [29]:
data = set()
for ids, val in factSegment.groupby('customer_id').aggregate({'amount':'sum'}).amount.to_dict().items():
    data.add((ids, val))
m_val = pd.DataFrame(data).rename(columns={0: "customer_id", 1:"monetary"})

factSegment = factSegment.merge(right=m_val, on='customer_id', how='inner')

* Frequency measure

In [30]:
data = set()
for ids, val in factSegment.groupby('customer_id').aggregate({'rental_id':'count'}).rental_id.to_dict().items():
    data.add((ids, val))
freq = pd.DataFrame(data).rename(columns={0: "customer_id", 1:"frequency"})

factSegment = factSegment.merge(right=freq, on='customer_id', how='inner')

In [31]:
print(factSegment.info())
factSegment.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16044 entries, 0 to 16043
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  16044 non-null  int64  
 1   city         16044 non-null  object 
 2   country      16044 non-null  object 
 3   active       16044 non-null  object 
 4   full_name    16044 non-null  object 
 5   rental_id    16044 non-null  int64  
 6   amount       16044 non-null  float64
 7   rental_date  16044 non-null  object 
 8   first_date   16044 non-null  object 
 9   recency      16044 non-null  int64  
 10  monetary     16044 non-null  float64
 11  frequency    16044 non-null  int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 1.5+ MB
None


,customer_id,city,country,active,full_name,rental_id,amount,rental_date,first_date,recency,monetary,frequency
0,1,Sasebo,Japan,1,MARY SMITH,1422,0.99,2005-06-15,2005-05-24,90,118.68,32
1,1,Sasebo,Japan,1,MARY SMITH,1476,9.99,2005-06-15,2005-05-24,90,118.68,32
2,1,Sasebo,Japan,1,MARY SMITH,1725,4.99,2005-06-16,2005-05-24,90,118.68,32
3,1,Sasebo,Japan,1,MARY SMITH,2308,4.99,2005-06-18,2005-05-24,90,118.68,32
4,1,Sasebo,Japan,1,MARY SMITH,2363,0.99,2005-06-18,2005-05-24,90,118.68,32


## Dump dimensions and fact to csv files

In [25]:
dimCustomer.to_csv('./data/tables/dimCustomer.csv', index_label='customer_key')
dimRental.to_csv('./data/tables/dimRental.csv', index_label='rental_key')
dimRental.to_csv('./data/tables/dimRental.txt', index_label='rental_key')
factSegment.to_csv('./data/tables/factSegment.csv')